# Task 1 - Apriori Algorithm for Recommender System (33 points)

**Task Definition:** In this programming assignment, you are required to implement the Apriori algorithm and apply it to mine frequent itemsets for recommendation. You are required to implement the algorithm from scratch by using only native Python libraries and NumPy. For efficiency you will need to convert the items to ids and sort them.

**Input:** The provided input file (`video_games.txt`) contains the favourite video games of 4096 users. Each line in the file corresponds to a user and represents a list of video games the user likes. An example:

*Torchlight 2;Transistor*

In the example above, the corresponding user likes the video games "Torchlight 2" and "Transistor".

**Output:** You need to implement the Apriori algorithm and use it to mine frequent itemsets. Set the relative minimum support to 0.055 and run the algorithm on the 4096 lists of video games. In other words, you need to extract all the itemsets that have an absolute support larger or equal to 226.

In [530]:
# TODO: uncomment the packages you used, please do not import additional non-native packages
# You may change the imports to the following format: from [package] import [class, method, etc.]

from collections import Counter
import itertools
import numpy as np

## 1.1 Loading the data and preprocessing (3 points)
**Task:** Solve the tasks explained in the TODOs and comments.

In [531]:
# TODO: read the data from the input file /data/video_games.txt (1 points)
with open('data/video_games.txt', "r") as file:
    data = file.readlines()

In [532]:
# TODO: determine the unique items and create a dictionary that maps an item to its id using enumerate (1 points)
unique_items = []
for line in data:
    items = line.strip().split(';')
    unique_items.extend(items)

unique_items = sorted(set(unique_items))

item_to_id = {item: id for id, item in enumerate(unique_items)}
id_to_item = {id: item for id, item in enumerate(unique_items)}

In [533]:
# TODO: map the items of the records to ids and sort each record (1 points)
mapped_records = []
for line in data:
    items = line.strip().split(';')
    mapped_items = [item_to_id[item] for item in items]
    mapped_items.sort()
    mapped_records.append(mapped_items)
# In the following tasks use the mapped records to compute the frequent itemsets.

## 1.2 Apriori algorithm (21 points)
### A) Determine the frequent 1 itemsets (3 points)
**Task:** Solve the tasks explained in the TODOs and comments.

In [534]:
# TODO: calculate the support of length-1 itemsets using Counter or defaultdict (1 points)
l1_items = Counter(item for record in mapped_records for item in record)

In [535]:
# TODO: filter out the frequent length-1 itemsets with their support (1 point)
frequent_l1_items = {item: support for item, support in l1_items.items() if support >= 226}

# TODO: save the length-1 frequent items to frequent_items with their support (1 points)
# Hint: Convert the itemsets to tuples or sets so that you can use them as keys in a dictionary.
frequent_items = {frozenset([item]): support for item, support in frequent_l1_items.items()}

### B) Determine the frequent n itemsets (15 points)
**Task:** Solve the tasks explained in the TODOs and comments.


In [536]:
def apriori_gen(itemsets):
    pruned_candidates = []
    for i, itemset1 in enumerate(itemsets):
        for j, itemset2 in enumerate(itemsets[i+1:]):
            # Join the itemsets if their first k-2 items are the same
            if itemset1[:-1] == itemset2[:-1]:
                # Prune the itemset by checking all subsets of length k-1
                new_itemset = itemset1 + [itemset2[-1]]
                subsets = [new_itemset[:i] + new_itemset[i+1:] for i in range(len(new_itemset))]
                if all(subset in itemsets for subset in subsets):
                    pruned_candidates.append(new_itemset)
    return pruned_candidates

In [537]:
# TODO: implement an algorithm to calculate the support of the given itemset (2 points)
# You do not need to implement a Hash Tree for calculating the supports.
def calculate_support(itemset):
    count = 0
    for record in mapped_records:
        if all(item in record for item in itemset):
            count += 1
    return count

In [538]:
# TODO: set the initial candidates which will be used to generate the frequent length-2 itemsets (1 point)
candidates = [[i] for i in range(len(unique_items))]

# TODO: set the correct loop condition (1 point)
while len(candidates) > 0:
    new_candidates = apriori_gen(candidates)
    supports = list(map(calculate_support, new_candidates))

    # TODO: filter out the frequent candidates (2 point)
    frequent_new_candidates = [itemset for itemset, support in zip(new_candidates, supports) if support >= 226]

    # TODO: add the frequent candidates to frequent_items (1 point)
    for itemset in frequent_new_candidates:
        frequent_items[tuple(itemset)] = calculate_support(itemset)

    # replace candidates with the new ones
    candidates = frequent_new_candidates

### C) Save your results (3 points)

**Task:** Save all the frequent itemsets along with their absolute supports into a text file named “patterns.txt” and place it in the root of your zip file. Every line corresponds to exactly one frequent itemset and should be in the following format:

*support:video_game_1;video_game_2;video_game_3;...*

For example, suppose an itemset (Fallout 4;Genshin Impact) has an absolute support 520, then the line corresponding to this frequent itemset in `patterns.txt` should be:

*520:Fallout 4;Genshin Impact*

In [539]:
with open("patterns.txt", "w") as file:
    for itemset, support in frequent_items.items():
        itemset_str = ";".join([id_to_item.get(item_id, "Unknown") for item_id in itemset])
        line = f"{support}:{itemset_str}\n"
        file.write(line)

## 1.3 Recommendation (9 points)

**Task:** Imagine you should recommend a video game to a user. You know that the user likes the video games "Elden Ring" and "Scarlet Nexus". Based on the result of the Apriori algorithm, implement an algorithm that gives a recommendation for this user by maximizing the confidence that the user will like the game. (6 points)

**Report:** Explain your choice and report the confidence score for your recommendation. (3 points)


In [540]:
def recommend_game(user_likes):
    with open("patterns.txt", "r") as file:
        frequent_items = {}
        for line in file:
            support, itemset = line.strip().split(":", 1)
            items = itemset.split(";")
            frequent_items[tuple(items)] = int(support)

    association_rules = []
    for antecedent, support in frequent_items.items():
        if set(user_likes).issubset(set(antecedent)):
            confidence = frequent_items[tuple(user_likes)] / support
            association_rules.append((antecedent, confidence))

    association_rules.sort(key=lambda x: x[1], reverse=True)
    recommended_games = [game for game in association_rules[0][0] if game not in user_likes]
    recommended_game = recommended_games

    return recommended_game

# Example usage
user_likes = ["Elden Ring","Scarlet Nexus"]
recommended_game = recommend_game(user_likes)
print(f"Recommended game: {recommended_game}")

Recommended game: ['Fallout 4']


External sources:

1. https://towardsdatascience.com/apriori-association-rule-mining-explanation-and-python-implementation-290b42afdfc6
2. https://chat.openai.com
3. https://www.simplilearn.com/tutorials/python-tutorial/enumerate-in-python#:~:text=Enumerate%20is%20a%20built%2Din,iterating%20over%20the%20iterable%20object.
4. https://github.com/nalinaksh/Association-Rule-Mining-Python/blob/master/apriori.py